# Data preprocessing to load into Neo4j

In [69]:
import pandas as pd
import os
import csv

## Custom Functions

In [70]:
def get_clean_list_of_files(paht):
    file_list = [file for file in os.listdir(paht) if not file.startswith(".")]
    file_list = [file for file in file_list if file.endswith(".csv")]
    return file_list

In [71]:
def process_relationships_neo4j(file_list, column_list, path_in, path_file_out, WRITE_DATA = False):
    
    countries = set()
    
    if os.path.exists(path_file_out) and WRITE_DATA:
        os.remove(path_file_out)
    
    with open(path_file_out, 'a') as f:
        
        first_line_file = [":START_ID(Country)",":END_ID(Country)","year:int","affected_number:float"]
        csv.writer(f).writerow(first_line_file)
    
        for file_in in file_list:
        
            df = pd.read_csv(path_in + file_in, skiprows=1, na_values=["*","0.0"])
            df = df[column_list]
            df = df.dropna(axis=0, how='any') # remove rows with missing data
            df['#affected+total'] = df['#affected+total'].astype(int)
            
            countries.update(df['#country+residence'].unique())
            countries.update(df['#country+origin'].unique())
            
            if WRITE_DATA:
                df.to_csv(f, header=False, index=False)
        
    return countries

In [94]:
def process_nodes_neo4j(file_list, path_in, path_file_out, countries_list, WRITE_DATA = False):
    
    if os.path.exists(path_file_out) and WRITE_DATA:
        os.remove(path_file_out)
    
    with open(path_file_out, 'a') as f:
        
        first_line_file = ["countryId:ID(Country)","year:int", "population:int", "int_migrant_stock", "pop_growth", "GDP_dollars", "inflation_percentage"]
        csv.writer(f).writerow(first_line_file)
    
        for file_in in file_list:
        
            df = pd.read_csv(path_in + file_in, skiprows=1, na_values=[""])
            
            country = df['#country+name'].unique()[0]
            
            if country in countries_list:
                
                df = df.dropna(axis=0, how='any') # remove rows with missing data
                
                return df
                
                
                
                
                if WRITE_DATA:
                    df_final.to_csv(f, header=False, index=False)
        
    return countries

## Common Variables

In [73]:
path_out = "../data/processed/neo4j/"
WRITE_DATA_FLAG = True

## People of concern. Residing
#### Relationship : **RESIDE_IN**

Define variables

In [74]:
path_in_residing = "../data/raw/residing/"
file_out_residing = path_out + 'relationsips_residing.csv'

In [75]:
column_list_residing = ['#country+origin', '#country+residence', '#date+year', "#affected+total"]

Define a list with all files located in the residing folder:

In [76]:
residing_list = get_clean_list_of_files(path_in_residing)
residing_list[0:4]

['refugees-residing-lie.csv',
 'refugees-residing-png.csv',
 'refugees-residing-fin.csv',
 'refugees-residing-cyp.csv']

Call function to process raw data and generate the required csv file to load into neo4j:

In [77]:
countries_residing = process_relationships_neo4j(residing_list, column_list_residing, path_in_residing, file_out_residing, WRITE_DATA_FLAG)

In [83]:
len(countries_residing)

224

In [ ]:
with open(path_out + 'countries_residing.csv', 'w') as f:
    f.write("countryId:ID(Country)\n")
    for item in countries:
        f.write('"' + item + '"\n')

## People of concern. Originating
#### Relationship : **ORIGINATE_FROM**

Define variables

In [78]:
path_in_originating = "../data/raw/originating/"
file_out_originating = path_out + 'relationsips_originating.csv'

In [79]:
column_list_originating = ['#country+residence', '#country+origin', '#date+year', "#affected+total"]

Define a list with all files located in the originating folder:

In [80]:
originating_list = get_clean_list_of_files(path_in_originating)
originating_list[0:4]

['refugees-originating-prt.csv',
 'refugees-originating-nzl.csv',
 'refugees-originating-khm.csv',
 'refugees-originating-isr.csv']

Call function to process raw data and generate the required csv file to load into neo4j:

In [82]:
countries_originating = process_relationships_neo4j(originating_list, column_list_originating, path_in_originating, file_out_originating, WRITE_DATA_FLAG)

In [84]:
len(countries_originating)

199

In [ ]:
with open(path_out + 'countries_originating.csv', 'w') as f:
    f.write("countryId:ID(Country)\n")
    for item in countries:
        f.write('"' + item + '"\n')

## World Bank Indicators

Define variables

In [88]:
path_in_indicators = "../data/raw/indicators/"
file_out_indicators_residing = path_out + 'countries_nodes_residing.csv'
file_out_indicators_originating = path_out + 'countries_nodes_originating.csv'

Define a list with all files located in the indicators folder:

In [89]:
indicators_list = get_clean_list_of_files(path_in_indicators)
indicators_list[0:4]

['world-bank-indicators-for-netherlands.csv',
 'world-bank-indicators-for-korea-rep.csv',
 'world-bank-indicators-for-antigua-and-barbuda.csv',
 'world-bank-indicators-for-gambia-the.csv']

Call function to process raw data and generate the required csv file to load into neo4j:

In [98]:
countries_indicators_residing = process_nodes_neo4j(indicators_list[0:1], path_in_indicators, file_out_indicators_residing, countries_residing, WRITE_DATA_FLAG)

In [99]:
countries_indicators_residing

,#country+name,#country+code,#date+year,#indicator+name,#indicator+code,#indicator+num
0,Netherlands,NLD,2017,"Population, total",SP.POP.TOTL,1.713285e+07
1,Netherlands,NLD,2016,"Population, total",SP.POP.TOTL,1.703031e+07
2,Netherlands,NLD,2015,"Population, total",SP.POP.TOTL,1.693992e+07
3,Netherlands,NLD,2014,"Population, total",SP.POP.TOTL,1.686501e+07
4,Netherlands,NLD,2013,"Population, total",SP.POP.TOTL,1.680443e+07
5,Netherlands,NLD,2012,"Population, total",SP.POP.TOTL,1.675496e+07
6,Netherlands,NLD,2011,"Population, total",SP.POP.TOTL,1.669307e+07
7,Netherlands,NLD,2010,"Population, total",SP.POP.TOTL,1.661539e+07
8,Netherlands,NLD,2009,"Population, total",SP.POP.TOTL,1.653039e+07
9,Netherlands,NLD,2008,"Population, total",SP.POP.TOTL,1.644559e+07


In [124]:
index_list = ["Population, total", 
              "International migrant stock (% of population)", 
              "Population growth (annual %)", 
              "GDP per capita, PPP (current international $)",
              "GINI index (World Bank estimate)",
              "Urban population (% of total)"]

In [125]:
df = countries_indicators_residing.pivot_table('#indicator+num', ['#country+name','#date+year'], '#indicator+name')

In [116]:
index_list.insert(0, '#country+name')

In [117]:
index_list.insert(1, '#date+year')

In [126]:
index_list

['Population, total',
 'International migrant stock (% of population)',
 'Population growth (annual %)',
 'GDP per capita, PPP (current international $)',
 'GINI index (World Bank estimate)',
 'Urban population (% of total)']

In [127]:
df = df[index_list]

In [128]:
df

#indicator+name           Population, total  \
#country+name #date+year                      
Netherlands   1960               11486631.0   
              1961               11638712.0   
              1962               11805689.0   
              1963               11965966.0   
              1964               12127120.0   
              1965               12294732.0   
              1966               12456251.0   
              1967               12598201.0   
              1968               12729721.0   
              1969               12877984.0   
              1970               13038526.0   
              1971               13194497.0   
              1972               13328593.0   
              1973               13439322.0   
              1974               13545056.0   
              1975               13666335.0   
              1976               13774037.0   
              1977               13856185.0   
              1978               13941700.0   
              1979               14038270.0   
              1980               14149800.0   
              1981               14247208.0   
              1982               14312690.0   
              1983               14367070.0   
              1984               14424211.0   
              1985               14491632.0   
              1986               14572278.0   
              1987               14665037.0   
              1988               14760094.0   
              1989               14848907.0   
              1990               14951510.0   
              1991               15069798.0   
              1992               15184166.0   
              1993               15290368.0   
              1994               15382838.0   
              1995               15459006.0   
              1996               15530498.0   
              1997               15610650.0   
              1998               15707209.0   
              1999               15812088.0   
              2000               15925513.0   
              2001               16046180.0   
              2002               16148929.0   
              2003               16225302.0   
              2004               16281779.0   
              2005               16319868.0   
              2006               16346101.0   
              2007               16381696.0   
              2008               16445593.0   
              2009               16530388.0   
              2010               16615394.0   
              2011               16693074.0   
              2012               16754962.0   
              2013               16804432.0   
              2014               16865008.0   
              2015               16939923.0   
              2016               17030314.0   
              2017               17132854.0   

#indicator+name           International migrant stock (% of population)  \
#country+name #date+year                                                  
Netherlands   1960                                                  NaN   
              1961                                                  NaN   
              1962                                                  NaN   
              1963                                                  NaN   
              1964                                                  NaN   
              1965                                                  NaN   
              1966                                                  NaN   
              1967                                                  NaN   
              1968                                                  NaN   
              1969                                                  NaN   
              1970                                                  NaN   
              1971                                                  NaN   
              1972                                                  NaN   
              1973                                    

In [ ]:
index_list = ["Population, total", 
              "International migrant stock (% of population)", 
              "Population growth (annual %)", 
              "GDP per capita, PPP (current international $)",
              "GINI index (World Bank estimate)",
              "Urban population (% of total)"]

In [137]:
df['New_ID'] = range(0, 0+len(df))

In [138]:
df

#indicator+name           Population, total  \
#country+name #date+year                      
Netherlands   1960                 11486631   
              1961                 11638712   
              1962                 11805689   
              1963                 11965966   
              1964                 12127120   
              1965                 12294732   
              1966                 12456251   
              1967                 12598201   
              1968                 12729721   
              1969                 12877984   
              1970                 13038526   
              1971                 13194497   
              1972                 13328593   
              1973                 13439322   
              1974                 13545056   
              1975                 13666335   
              1976                 13774037   
              1977                 13856185   
              1978                 13941700   
              1979                 14038270   
              1980                 14149800   
              1981                 14247208   
              1982                 14312690   
              1983                 14367070   
              1984                 14424211   
              1985                 14491632   
              1986                 14572278   
              1987                 14665037   
              1988                 14760094   
              1989                 14848907   
              1990                 14951510   
              1991                 15069798   
              1992                 15184166   
              1993                 15290368   
              1994                 15382838   
              1995                 15459006   
              1996                 15530498   
              1997                 15610650   
              1998                 15707209   
              1999                 15812088   
              2000                 15925513   
              2001                 16046180   
              2002                 16148929   
              2003                 16225302   
              2004                 16281779   
              2005                 16319868   
              2006                 16346101   
              2007                 16381696   
              2008                 16445593   
              2009                 16530388   
              2010                 16615394   
              2011                 16693074   
              2012                 16754962   
              2013                 16804432   
              2014                 16865008   
              2015                 16939923   
              2016                 17030314   
              2017                 17132854   

#indicator+name           International migrant stock (% of population)  \
#country+name #date+year                                                  
Netherlands   1960                                                  NaN   
              1961                                                  NaN   
              1962                                                  NaN   
              1963                                                  NaN   
              1964                                                  NaN   
              1965                                                  NaN   
              1966                                                  NaN   
              1967                                                  NaN   
              1968                                                  NaN   
              1969                                                  NaN   
              1970                                                  NaN   
              1971                                                  NaN   
              1972                                                  NaN   
              1973                                    

In [139]:
with open(path_out + 'countries_residing.csv', 'w') as f:
    
    first_line_file = ["country:LABEL(Country)",
                       "year:int", 
                       "population:int", 
                       "int_migrant_stock:float", 
                       "pop_growth_percentage:float", 
                       "GDP_dollars:float", 
                       "GINI:float", 
                       "urban_pop_percentage:float",
                       ":ID"]
    csv.writer(f).writerow(first_line_file)
    df['Population, total'] = df['Population, total'].astype(int)
    df.to_csv(f, header=False)

In [136]:
countries_indicators_originating = process_nodes_neo4j(indicators_list, path_in_indicators, file_out_indicators_originating, countries_originating, WRITE_DATA_FLAG)